In [1]:
# !pip install numpy==1.19.5 tensorflow==2.5.0 keras==2.5.0rc0 scikit-learn==1.1.2 matplotlib

In [2]:
# !git config --global core.compression 0
# !git clone https://github.com/ZPZhou-lab/tfkan.git
# !cd tfkan && pip install . && cd ..

In [3]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split

from models import CNN_1D_model, CNN_1D_BN_model, KAN_CNN_1D_model, KAN_CNN_1D_BN_model
from data_loader import load_data



def new_model(model_type="conv1d"):
    if model_type == "regular":
        model = CNN_1D_model()
    elif model_type == "bn":
        model = CNN_1D_BN_model()
    elif model_type == "kan":
        model = KAN_CNN_1D_model()
    elif model_type == "kan-bn":
        model = KAN_CNN_1D_BN_model()
    else:
        raise ValueError(f"Invalid model_type {model_type}")
        
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


def split_data(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)
    x_train, x_val, y_train, y_val = train_test_split(
        x_train, y_train, test_size=0.25, stratify=y_train
    )
    return x_train, y_train, x_val, y_val, x_test, y_test


def save_results(data, filename):
    with open(f"results/{filename}.json", "w") as json_file:
        json.dump(data, json_file, indent=4)


def load_results(filename):
    with open(f"results/{filename}.json", "r") as json_file:
        data = json.load(json_file)
    return data



individuals = [
    "robert_sajina",
    "romeo_sajina",
    "alesandro_zuzic",
    "luka_blaskovic",
    "laura_loncaric",
    "matej_visnjic",
    "matija_hamer",
    "rafael_krstacic",
]

ratios = [0.001, 0.005, 0.01, 0.025, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
MODEL_TYPE = "kan-bn"

In [5]:

# Individual training
for num_repeat in range(5):
    results = {}
    for train_ratio in ratios:
        results[train_ratio] = {}
        for individual in individuals:
            x, y = load_data(
                [f"data/{individual}"], labels=["left", "right", "jump", "none"], shape=(60, 16)
            )

            x_train, y_train, x_val, y_val, x_test, y_test = split_data(x, y)
            if train_ratio < 1:
                x_train, _, y_train, _ = train_test_split(
                    x_train, y_train, test_size=1 - train_ratio, stratify=y_train
                )

            model = new_model(MODEL_TYPE)
            es = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
            history = model.fit(
                x_train,
                y_train,
                epochs=1000,
                batch_size=64,
                validation_data=(x_val, y_val),
                callbacks=[es],
                verbose=0,
            )

            results[train_ratio][individual] = {
                "iterations": len(history.epoch) * history.params["steps"],
                "test_acc": model.evaluate(x_test, y_test, verbose=0)[1],
            }
            # print(individual, '\t', len(history.epoch), '\t', round(model.evaluate(x_test, y_test, verbose=0)[1], 4))
        print(f"Done for {train_ratio}", results[train_ratio])

    save_results(results, f"{MODEL_TYPE}_results_individual_{num_repeat}")

Done for 0.001 {'robert_sajina': {'iterations': 22, 'test_acc': 0.3793981373310089}, 'romeo_sajina': {'iterations': 85, 'test_acc': 0.6467592716217041}, 'alesandro_zuzic': {'iterations': 28, 'test_acc': 0.5361111164093018}, 'luka_blaskovic': {'iterations': 36, 'test_acc': 0.4576388895511627}, 'laura_loncaric': {'iterations': 23, 'test_acc': 0.3567129671573639}, 'matej_visnjic': {'iterations': 22, 'test_acc': 0.37291666865348816}, 'matija_hamer': {'iterations': 32, 'test_acc': 0.43101853132247925}, 'rafael_krstacic': {'iterations': 33, 'test_acc': 0.4650462865829468}}
Done for 0.005 {'robert_sajina': {'iterations': 148, 'test_acc': 0.7282407283782959}, 'romeo_sajina': {'iterations': 193, 'test_acc': 0.8773148059844971}, 'alesandro_zuzic': {'iterations': 79, 'test_acc': 0.6574074029922485}, 'luka_blaskovic': {'iterations': 59, 'test_acc': 0.646064817905426}, 'laura_loncaric': {'iterations': 90, 'test_acc': 0.46990740299224854}, 'matej_visnjic': {'iterations': 231, 'test_acc': 0.643981456

Done for 0.005 {'robert_sajina': {'iterations': 95, 'test_acc': 0.6543981432914734}, 'romeo_sajina': {'iterations': 429, 'test_acc': 0.8782407641410828}, 'alesandro_zuzic': {'iterations': 92, 'test_acc': 0.6907407641410828}, 'luka_blaskovic': {'iterations': 61, 'test_acc': 0.6949074268341064}, 'laura_loncaric': {'iterations': 5, 'test_acc': 0.25578704476356506}, 'matej_visnjic': {'iterations': 206, 'test_acc': 0.6902777552604675}, 'matija_hamer': {'iterations': 141, 'test_acc': 0.6837962865829468}, 'rafael_krstacic': {'iterations': 66, 'test_acc': 0.6886574029922485}}
Done for 0.01 {'robert_sajina': {'iterations': 114, 'test_acc': 0.7935185432434082}, 'romeo_sajina': {'iterations': 111, 'test_acc': 0.8564814925193787}, 'alesandro_zuzic': {'iterations': 96, 'test_acc': 0.7662037014961243}, 'luka_blaskovic': {'iterations': 81, 'test_acc': 0.7111111283302307}, 'laura_loncaric': {'iterations': 222, 'test_acc': 0.4523148238658905}, 'matej_visnjic': {'iterations': 84, 'test_acc': 0.671527802

Done for 0.01 {'robert_sajina': {'iterations': 111, 'test_acc': 0.7145833373069763}, 'romeo_sajina': {'iterations': 78, 'test_acc': 0.8458333611488342}, 'alesandro_zuzic': {'iterations': 111, 'test_acc': 0.7550926208496094}, 'luka_blaskovic': {'iterations': 78, 'test_acc': 0.7467592358589172}, 'laura_loncaric': {'iterations': 48, 'test_acc': 0.3513889014720917}, 'matej_visnjic': {'iterations': 102, 'test_acc': 0.7393518686294556}, 'matija_hamer': {'iterations': 93, 'test_acc': 0.7152777910232544}, 'rafael_krstacic': {'iterations': 87, 'test_acc': 0.7168981432914734}}
Done for 0.025 {'robert_sajina': {'iterations': 354, 'test_acc': 0.8849536776542664}, 'romeo_sajina': {'iterations': 276, 'test_acc': 0.9388889074325562}, 'alesandro_zuzic': {'iterations': 384, 'test_acc': 0.8773148059844971}, 'luka_blaskovic': {'iterations': 348, 'test_acc': 0.9104166626930237}, 'laura_loncaric': {'iterations': 60, 'test_acc': 0.3229166567325592}, 'matej_visnjic': {'iterations': 378, 'test_acc': 0.8645833

Done for 0.025 {'robert_sajina': {'iterations': 312, 'test_acc': 0.8914352059364319}, 'romeo_sajina': {'iterations': 300, 'test_acc': 0.9337962865829468}, 'alesandro_zuzic': {'iterations': 282, 'test_acc': 0.8636574149131775}, 'luka_blaskovic': {'iterations': 444, 'test_acc': 0.9067129492759705}, 'laura_loncaric': {'iterations': 114, 'test_acc': 0.40231481194496155}, 'matej_visnjic': {'iterations': 420, 'test_acc': 0.9238426089286804}, 'matija_hamer': {'iterations': 384, 'test_acc': 0.8430555462837219}, 'rafael_krstacic': {'iterations': 426, 'test_acc': 0.893750011920929}}
Done for 0.05 {'robert_sajina': {'iterations': 517, 'test_acc': 0.9743055701255798}, 'romeo_sajina': {'iterations': 693, 'test_acc': 0.9974536895751953}, 'alesandro_zuzic': {'iterations': 495, 'test_acc': 0.9523147940635681}, 'luka_blaskovic': {'iterations': 781, 'test_acc': 0.9733796119689941}, 'laura_loncaric': {'iterations': 143, 'test_acc': 0.4798611104488373}, 'matej_visnjic': {'iterations': 781, 'test_acc': 0.9

Done for 0.05 {'robert_sajina': {'iterations': 495, 'test_acc': 0.9761574268341064}, 'romeo_sajina': {'iterations': 869, 'test_acc': 0.9956018328666687}, 'alesandro_zuzic': {'iterations': 495, 'test_acc': 0.9375}, 'luka_blaskovic': {'iterations': 517, 'test_acc': 0.9747685194015503}, 'laura_loncaric': {'iterations': 253, 'test_acc': 0.41620370745658875}, 'matej_visnjic': {'iterations': 924, 'test_acc': 0.978935182094574}, 'matija_hamer': {'iterations': 693, 'test_acc': 0.9851852059364319}, 'rafael_krstacic': {'iterations': 627, 'test_acc': 0.9664351940155029}}
Done for 0.1 {'robert_sajina': {'iterations': 1260, 'test_acc': 0.9923611283302307}, 'romeo_sajina': {'iterations': 1071, 'test_acc': 1.0}, 'alesandro_zuzic': {'iterations': 1176, 'test_acc': 0.9833333492279053}, 'luka_blaskovic': {'iterations': 1029, 'test_acc': 0.9995370507240295}, 'laura_loncaric': {'iterations': 315, 'test_acc': 0.49699074029922485}, 'matej_visnjic': {'iterations': 1323, 'test_acc': 0.9960648417472839}, 'mati

In [6]:

# Fine-tuning

for num_repeat in range(5):
    results_ft = {k: {} for k in ratios}

    for individual in individuals:
        x, y = load_data(
            [f"data/{inv}" for inv in individuals if inv != individual],
            labels=["left", "right", "jump", "none"],
            shape=(60, 16),
        )
        x_all, y_all, x_val_all, y_val_all, x_test_all, y_test_all = split_data(x, y)
        model = new_model(MODEL_TYPE)
        es = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
        history = model.fit(
            x_all,
            y_all,
            epochs=1000,
            batch_size=64,
            validation_data=(x_val_all, y_val_all),
            callbacks=[es],
            verbose=0,
        )
        print("Global acc:", model.evaluate(x_test_all, y_test_all, verbose=0)[1])

        x_ind, y_ind = load_data(
            [f"data/{individual}"], labels=["left", "right", "jump", "none"], shape=(60, 16)
        )

        x_train, y_train, x_val, y_val, x_test, y_test = split_data(x_ind, y_ind)
        for train_ratio in ratios:
            ind_x_train, ind_y_train = x_train, y_train
            if train_ratio < 1:
                ind_x_train, _, ind_y_train, _ = train_test_split(
                    x_train, y_train, test_size=1 - train_ratio, stratify=y_train
                )
            ind_model = new_model(MODEL_TYPE)
            ind_model.build(input_shape=[None] + list(ind_x_train.shape[1:]))
            ind_model.set_weights(model.get_weights())
            es = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
            history = ind_model.fit(
                ind_x_train,
                ind_y_train,
                epochs=1000,
                batch_size=64,
                validation_data=(x_val, y_val),
                callbacks=[es],
                verbose=0,
            )

            results_ft[train_ratio][individual] = {
                "iterations": len(history.epoch) * history.params["steps"],
                "test_acc": ind_model.evaluate(x_test, y_test, verbose=0)[1],
            }
        print(f"Done for {individual}")
    save_results(results_ft, f"{MODEL_TYPE}_results_ft_{num_repeat}")

Global acc: 0.9022986888885498
Done for robert_sajina
Global acc: 0.8576153516769409
Done for romeo_sajina
Global acc: 0.8764346241950989
Done for alesandro_zuzic
Global acc: 0.8982966542243958
Done for luka_blaskovic
Global acc: 0.9992392659187317
Done for laura_loncaric
Global acc: 0.9544236660003662
Done for matej_visnjic
Global acc: 0.8182900547981262
Done for matija_hamer
Global acc: 0.9282950162887573
Done for rafael_krstacic
Global acc: 0.8939970135688782
Done for robert_sajina
Global acc: 0.9233669638633728
Done for alesandro_zuzic
Global acc: 0.9542252421379089
Done for luka_blaskovic
Global acc: 0.9977840185165405
Done for laura_loncaric
Global acc: 0.8961799144744873
Done for matej_visnjic
Global acc: 0.9779725670814514
Done for matija_hamer
Global acc: 0.9744336009025574
Done for rafael_krstacic
Global acc: 0.9059037566184998
Done for robert_sajina
Global acc: 0.756308913230896
Done for romeo_sajina
Global acc: 0.9713577032089233
Done for alesandro_zuzic
Global acc: 0.95405

In [7]:
# Transfer learning
for num_repeat in range(5):

    results_tl = {k: {} for k in ratios}

    for individual in individuals:
        x, y = load_data(
            [f"data/{inv}" for inv in individuals if inv != individual],
            labels=["left", "right", "jump", "none"],
            shape=(60, 16),
        )
        x_all, y_all, x_val_all, y_val_all, x_test_all, y_test_all = split_data(x, y)
        model = new_model(MODEL_TYPE)
        es = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
        history = model.fit(
            x_all,
            y_all,
            epochs=1000,
            batch_size=64,
            validation_data=(x_val_all, y_val_all),
            callbacks=[es],
            verbose=0,
        )
        print("Global acc:", model.evaluate(x_test_all, y_test_all, verbose=0)[1])

        x_ind, y_ind = load_data(
            [f"data/{individual}"], labels=["left", "right", "jump", "none"], shape=(60, 16)
        )

        x_train, y_train, x_val, y_val, x_test, y_test = split_data(x_ind, y_ind)
        for train_ratio in ratios:
            ind_x_train, ind_y_train = x_train, y_train
            if train_ratio < 1:
                ind_x_train, _, ind_y_train, _ = train_test_split(
                    x_train, y_train, test_size=1 - train_ratio, stratify=y_train
                )

            ind_model = new_model(MODEL_TYPE)
            ind_model.build(input_shape=[None] + list(ind_x_train.shape[1:]))
            ind_model.set_weights(model.get_weights())
            for layer in ind_model.layers[:-1]:
                layer.trainable = False
            ind_model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"],
            )

            es = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
            history = ind_model.fit(
                ind_x_train,
                ind_y_train,
                epochs=1000,
                batch_size=64,
                validation_data=(x_val, y_val),
                callbacks=[es],
                verbose=0,
            )

            results_tl[train_ratio][individual] = {
                "iterations": len(history.epoch) * history.params["steps"],
                "test_acc": ind_model.evaluate(x_test, y_test, verbose=0)[1],
            }
        print(f"Done for {individual}")
    save_results(results_tl, f"{MODEL_TYPE}_results_tl_{num_repeat}")

Global acc: 0.9613031148910522
Done for robert_sajina
Global acc: 0.9446337223052979
Done for romeo_sajina
Global acc: 0.8809326887130737
Done for alesandro_zuzic
Global acc: 0.9353398084640503
Done for luka_blaskovic
Global acc: 0.9982801675796509
Done for laura_loncaric
Global acc: 0.9464858770370483
Done for matej_visnjic
Global acc: 0.8177939653396606
Done for matija_hamer
Global acc: 0.9551513195037842
Done for rafael_krstacic
Global acc: 0.8552670478820801
Done for robert_sajina
Global acc: 0.8655201196670532
Done for romeo_sajina
Global acc: 0.9201256632804871
Done for alesandro_zuzic
Global acc: 0.9558458924293518
Done for luka_blaskovic
Global acc: 0.9991731643676758
Done for laura_loncaric
Global acc: 0.9617000222206116
Done for matej_visnjic
Global acc: 0.9177112579345703
Done for matija_hamer
Global acc: 0.9718207120895386
Done for rafael_krstacic
Global acc: 0.9215478897094727
Done for robert_sajina
Global acc: 0.9277989268302917
Done for romeo_sajina
Global acc: 0.9285265